In [11]:
# import relevant packages

import pandas as pd
import os
import re

from html import unescape
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)

## Load raw datasets

In [12]:
df_dict = dict()

PATH = "../0_data/main/0_raw"

for file in sorted(os.listdir(PATH)):
    if "ipynb" not in file:
        print(re.sub('\.csv$', '', file))
        df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/{file}")

<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
/var/folders/31/pb5329nd0sl5sgm6rlw3p6340000gn/T/ipykernel_32419/2220965070.py:7: SyntaxWarning: invalid escape sequence '\.'
  print(re.sub('\.csv$', '', file))
/var/folders/31/pb5329nd0sl5sgm6rlw3p6340000gn/T/ipykernel_32419/2220965070.py:8: SyntaxWarning: invalid escape sequence '\.'
  df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/{file}")


bas19_es
dyn21_en
for19_pt
fou18_en
has19_hi
has20_hi
has21_hi
ken20_en
ous19_ar
ous19_fr
san20_it


## Reformat columns
Need separate logic for different datasets. 1 is for hateful, 0 for non-hateful.

In [13]:
# Dynabench 2021 / English
df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)

# Founta 2018 / English
df_dict["fou18_en"].label.replace({'hateful': 1, "abusive": 0, "normal": 0, "spam": 0}, inplace = True)

# Kennedy 2020 / English
df_dict["ken20_en"].rename(columns={"label_hate_maj": "label"}, inplace=True)

# Fortuna 2019 / Portuguese
df_dict["for19_pt"].rename(columns={"hatespeech_comb": "label"}, inplace=True)

# Basile 2019 / Spanish
df_dict["bas19_es"].rename(columns={"HS": "label"}, inplace=True)

# Sanguinetti 2020 / Italian
df_dict["san20_it"].rename(columns={"hs": "label"}, inplace=True)

# Ousidhoum 2019 / Arabic & French
for d in ["ous19_ar", "ous19_fr"]:
    df_dict[d]["label"] = df_dict[d].sentiment.apply(lambda x: 1 if "hateful" in x else 0)
    # text was already cleaned in a way that conflicts with our later cleaning, so we align it here
    df_dict[d]["text"] = df_dict[d].tweet.apply(lambda x: x.replace("@url", "http"))
    
# HASOC 19, 20 and 21 / Hindi
for d in ["has19_hi", "has20_hi", "has21_hi"]:
    df_dict[d]["label"] = df_dict[d].task_2.apply(lambda x: 1 if x=="HATE" else 0)
    
# drop redundant columns
for dataset in df_dict:
    if "split" in df_dict[dataset].columns:
        df_dict[dataset] = df_dict[dataset][["text", "label", "split"]]
    else:
        df_dict[dataset] = df_dict[dataset][["text", "label"]]

/var/folders/31/pb5329nd0sl5sgm6rlw3p6340000gn/T/ipykernel_32419/1583237372.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)
/var/folders/31/pb5329nd0sl5sgm6rlw3p6340000gn/T/ipykernel_32419/1583237372.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_s

## Clean text

In [14]:
def clean_text(text):
    text = unescape(text)
    text = re.sub(r"@[A-Za-z0-9_-]+",'@user',text) # format expected by XLM-T
    text = re.sub(r"http\S+",'http',text) # format expected by XLM-T
    text = re.sub(r"\n",' ',text)
    text = re.sub(r"\t",' ',text)
    text = text.replace("[URL]", "http") # format expected by XLM-T
    text = text.strip()
    return text

for dataset in df_dict:
    df_dict[dataset]["text"] = df_dict[dataset].text.apply(lambda x: clean_text(x))

## Boost proportion of hate in English datasets to match Dynabench
We are doing a first phase of fine-tuning on up to 20k entries in Dynabench. Dynabench has ca. 53% hate. The other English datasets have a lower proportion of hate so we drop non-hate to make the proportion more comparable at 20k entries.

In [15]:
# boost Kennedy 2020 / English to have 50% hate (up from ca. 30%)
df_dict["ken20_en"] = pd.concat([df_dict["ken20_en"][df_dict["ken20_en"].label==1], df_dict["ken20_en"][df_dict["ken20_en"].label==0].sample(11596, random_state=123)]).sample(frac=1, random_state=123)

# boost Founta 2018 / English to have 22% hate, which is max possible (up from ca. 5%)
df_dict["fou18_en"] = pd.concat([df_dict["fou18_en"][df_dict["fou18_en"].label==1], df_dict["fou18_en"][df_dict["fou18_en"].label==0].sample(17600, random_state=123)]).sample(frac=1, random_state=123)


## Show descriptive stats

In [16]:
def descriptive_stats(df):
    n_total = df.shape[0]
    n_hate = df.label.sum()
    print("{} entries, of which {} ({:.2%}) are hateful.".format(n_total, n_hate, n_hate/n_total))
    return df.label.sum()/len(df), len(df)

for dataset in df_dict:
    print(dataset.upper())
    descriptive_stats(df_dict[dataset])
    display(df_dict[dataset].sample(5))
    print()

BAS19_ES
6600 entries, of which 2739 (41.50%) are hateful.


,text,label
2438,@user Me flipa😍 Me tienes que enseñar a hacer pelo me cago en la puta JAJAJA,0
4938,¿Que harian los hombres sin las mujeres? Pues domesticar otro animal y esta vez no enseñarle a hablar.,1
295,"Que no es presunto 🐍 @user que ESCORIA😈 es CULPABLE del ASESINATO,DESAPARICION,SECUESTRO Y VIOLACIÓN de #DianaQuer angelico 😓🌷💕🐾🌛 D.E.P que este ESCORIA😈q su trastorno solo es #HijoPutismoAgudo va a pagarlo 😈😈 #TodasSomosDiana 💕🐾🌛",0
2765,Hasta ahora me vi el capítulo de #TWD y solo tengo que decir que Eugene es un hijo de puta que merecía morir en lugar de Carl. Ah y qué adoro a Maggie obviamente no más que a Carol.,0
3865,El curioso caso del hombre que solo ve machismo si el acusado es un árabe.,0



DYN21_EN
41144 entries, of which 22175 (53.90%) are hateful.


,text,label,split
22247,"You see, children imitate society and we live in a time where falsehoods are embraced in the name of pushing a narrative. Equality of opportunity? Where?",0,test
22568,All women and men and others are looking for is someone to provide for us.,0,dev
18435,who is truly suffering? Those of you who need to be lead by leaders or those who are waiting for the next hand out jerked off expired food ?,0,train
8394,I am a big fat curry muncher and would eat it over anything!,0,train
8332,Today I feel like I have to mention how muslims are influencing the level of wonderful things in the world. By that I mean negatively of course since muslims are basically just pigs to lookat,1,train



FOR19_PT
5670 entries, of which 1788 (31.53%) are hateful.


...

,text,label
1221,@user @user اخر من يتكلم انتم ياخونة ولالكم دار ولا بلد انتم خنازير الغرب وطهران والشرف والغير… http,0
1794,اي امرأة متبرجة نازعة حجابها لاتشرفنا كا رجال وليس ذكور ولن نفكر نرتبط بها اشوفها نازعه حجابها كمية… http,0
3186,@user يا حقير هيدا العهد يلي انت بدك يلي انت عّم تبوس راص عون يا واطي يا خنزير,1
2381,تخيل بنفس الوقت تكلم خنزير بالواقع ويسبك سبودر بالخاص ويمنشنك وسيف ؟ اي انحطاط توصلت له يا انا,0
2017,ونهايتكم قادمة . شاربي بول البعير .,1



OUS19_FR
4014 entries, of which 399 (9.94%) are hateful.


,text,label
1405,@user @user un adepte du lieu commun gauchiste,0
2827,bravo au réalisateur de #lejouroujaibrulemoncoeur qui inclut le #harcèlement #antisemite dans le film « #sale #juif ». une réalité.,0
2796,: [l'africain d'algérie est souvent raciste sur twitter.,0
2734,une gauchiste reste une gauchiste. et pourtant elle bataillé. http,0
1470,maitena biraben.. la rumeur raconte qu en bonne gauchiste elle prévoit de reverser l'intégralité de la somme à des… http,0



SAN20_IT
8100 entries, of which 3388 (41.83%) are hateful.


,text,label,split
4312,Vanno affondate. Hanno rotto i c.....i Aquarius vuol dettare ancora legge: carica migranti e rifiuta gli ordini libici http,1,train
4079,"@user Mi chiedo come sia possibile definire Islam religione di pace invece di mettere in guardia il mondo da questa ""cultura"" subumana #sposebambine",1,train
7379,"Business immigrati Tanti ci guadagnano: Le ONG che trasportano, Le Coop che ospitano, il caporalato, a breve l'industria con mano d'opera a basso costo, i Sindacati che da loro possono ripartire a vendere tessere, i partiti pro immigrazione che prenderanno i loro voti.",0,test
6723,"Se stranieri, delinquenti. Se italiani, derubricata a ragazzata e in premio non c'è la medaglia di Salvini. É l' Italietta legastellata.",0,train
4355,"Scusate, ma a proposito di #facciamocome: non potremmo semplicemente espellere anche noi tutti gli stranieri disoccupati? (così evitiamo anche il groviglio della discriminazione per il rdc)",1,train


## Create and export splits

In [17]:
# set aside 2k from each dataset for testing and 500 for dev
# except for Ousidhoum in French and Arabic, where train set would otherwise be too small
# and for HASOC 20 and 21 in Hindi, where test splits are given

TEST_SIZE = 2000
DEV_SIZE = 500

for dataset in df_dict:
    if "ous19_fr" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 2000, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1500.csv", index=False)
    elif "ous19_ar" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 1300, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1000, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_300.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1000.csv", index=False)
    elif "has19_hi" in dataset or "has20_hi" in dataset: # use provided test sets
        df_dict[dataset][df_dict[dataset]["split"]=="test"].to_csv(f"../0_data/main/1_clean/{dataset}/test_{len(df_dict[dataset][df_dict[dataset]['split']=='test'])}.csv", index=False)
        df_dict[dataset], devset = train_test_split(df_dict[dataset][df_dict[dataset]["split"]=="train"], test_size = 500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
    else:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = TEST_SIZE+DEV_SIZE, random_state=123)
        devset, testset = train_test_split(devtest, test_size = TEST_SIZE, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_{DEV_SIZE}.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_{TEST_SIZE}.csv", index=False)
        
# export all data that is not test or dev, so we can use it for full sample training
for dataset in df_dict:
    df_dict[dataset].to_csv(f"../0_data/main/1_clean/{dataset}/train_{len(df_dict[dataset])}.csv", index=False)

In [18]:
# create differently-sized train portions from rest of data

SEEDS = 10 # for repeated experiments with different random data selection
N_RANGE = [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000]

for dataset in df_dict:
    print(dataset.upper())
    for n in N_RANGE:
        
        # save all splits for English test sets
        if n<len(df_dict[dataset]) and ("dyn21" in dataset or "ken20" in dataset or "fou18" in dataset): 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
        
        # save splits up to 2k for other datasets
        elif n<len(df_dict[dataset]) and n<=2000: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):  
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
    
    print()

BAS19_ES
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set

DYN21_EN
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set
  saving n = 3000 training set
  saving n = 4000 training set
  saving n = 5000 training set
  saving n = 10000 training set
  saving n = 20000 training set

FOR19_PT
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 